In [1]:
# -*- coding: utf-8 -*-
"""bus_stops_extraction.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1OikJekLXp81Irr14GkLemhqGEyvsEXlh

Importing python libraries
"""

import pandas as pd
import numpy as np
from datetime import datetime,date, timedelta

! pip install geopandas
import geopandas as gpd
from geopandas import GeoDataFrame as gdf


import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

#Importing 
# 1. raw GPS data
# 2. splitted trip data from bus_trip_extraction.py
# 3. bus stops details (Latitude and longitude)
raw_files = [
    '../Data Sets/Raw GPS data Kandy Digana Buses/digana_2021_10.csv',
    # '../Data Sets\Raw GPS data Kandy Digana Buses/digana_2021_11.csv',
    # '../Data Sets\Raw GPS data Kandy Digana Buses/digana_2021_12.csv',
    # '../Data Sets\Raw GPS data Kandy Digana Buses/digana_2022_01.csv',
    # '../Data Sets\Raw GPS data Kandy Digana Buses/digana_2022_02.csv',
    # '../Data Sets\Raw GPS data Kandy Digana Buses/digana_2022_07.csv',
    # '../Data Sets\Raw GPS data Kandy Digana Buses/digana_2022_08.csv',
    # '../Data Sets\Raw GPS data Kandy Digana Buses/digana_2022_09.csv',
    # '../Data Sets\Raw GPS data Kandy Digana Buses/digana_2022_10.csv'
]


path_trip_ends = '../DataOut/trip_ends.csv'
path_bus_trips = '../DataOut/bus_trips.csv'
path_bus_stops = '../Data Sets/bus_stops_and_terminals_654.csv'

trip_ends = pd.read_csv(path_trip_ends)
bus_trips = pd.read_csv(path_bus_trips)
bus_stops= pd.read_csv(path_bus_stops)

# Create an empty list to store DataFrames
dfs = []

# Loop through the list of file names and read each CSV into a DataFrame
for file in raw_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate the DataFrames vertically
raw_data = pd.concat(dfs, ignore_index=True)


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def raw_data_cleaning(raw_data):
    """
    Removal of records with error records.
    Remove data with zero values for longitude and latitude columns.
    Remove data with dates outside the desired range.
    Sort data by time and device.

    Args:
        raw_data (pd.DataFrame): Crude raw GPS data filtered out from the server for the required time window.

    Returns:
        gps_data (pd.DataFrame): A cleaned dataframe object of GPS data.
    """

    # raw_data = raw_data.drop(drop_columns, axis=1)

    gps_data = raw_data[raw_data.latitude != 0]
    gps_data = gps_data[gps_data.longitude != 0]  # cleaning zero values for latitude & longitude

    gps_data['date'] = pd.to_datetime(gps_data['devicetime']).dt.date  # split date and time separately into datetime variables
    gps_data['time'] = pd.to_datetime(gps_data['devicetime']).dt.time

    # Remove rows with dates outside the desired range
    start_date = pd.Timestamp("2021-10-01").date()
    end_date = pd.Timestamp("2022-10-31").date()
    gps_data = gps_data[(gps_data['date'] >= start_date) & (gps_data['date'] <= end_date)]

    gps_data = gps_data.sort_values(['deviceid', 'date', 'time'])  # sorting dataset by time and device

    return gps_data


raw_data.columns

additional_columns = ['servertime','fixtime','address','routeid']

# drop the additional columns
raw_data = raw_data.drop(additional_columns, axis = 1)



gps_data= raw_data_cleaning(raw_data)

In [3]:
gps_data

,id,deviceid,devicetime,latitude,longitude,speed,date,time
281253,574073556,116,2021-10-15 11:37:14,7.293917,80.736137,0.0,2021-10-15,11:37:14
281400,574078368,116,2021-10-15 11:39:26,7.294845,80.735427,0.0,2021-10-15,11:39:26
281492,574082837,116,2021-10-15 11:39:41,7.294825,80.735470,0.0,2021-10-15,11:39:41
281493,574082838,116,2021-10-15 11:39:56,7.294817,80.735472,0.0,2021-10-15,11:39:56
281494,574082840,116,2021-10-15 11:40:11,7.294813,80.735470,0.0,2021-10-15,11:40:11
...,...,...,...,...,...,...,...,...
608425,587300473,1377,2021-10-27 19:59:36,7.263888,80.700603,0.0,2021-10-27,19:59:36
608426,587300474,1377,2021-10-27 19:59:51,7.263888,80.700603,0.0,2021-10-27,19:59:51
635647,589032240,1377,2021-10-27 20:00:06,7.263888,80.700603,0.0,2021-10-27,20:00:06
556115,585366438,1377,2021-10-27 20:00:09,7.263888,80.700603,0.0,2021-10-27,20:00:09


In [4]:
bus_stops

,stop_id,route_id,direction,address,latitude,longitude
0,BT01,654,Kandy-Digana,Kandy,7.292462,80.634978
1,101,654,Kandy-Digana,Wales Park,7.291186,80.637662
2,102,654,Kandy-Digana,Mahamaya,7.287840,80.645840
3,103,654,Kandy-Digana,Lewella junction,7.294430,80.650030
4,104,654,Kandy-Digana,Talwatta,7.286701,80.660336
5,105,654,Kandy-Digana,Tennekumbura Bridge,7.281866,80.666030
6,106,654,Kandy-Digana,Kalapura Junction Busstop,7.279830,80.676210
7,107,654,Kandy-Digana,Nattarampotha Junction Bus Stop,7.279117,80.679450
8,108,654,Kandy-Digana,Kundasale New town,7.280900,80.684160
9,109,654,Kandy-Digana,Warapitiya,7.281490,80.686350


In [5]:
#developing geo-buffer rings around every bus stops
def bus_stop_buffer_create(gps_data,bus_stops,stop_buffer,extra_buffer):

  """
  
    Buffer and additional buffer  created  to accomodate points if they were missed in standard stop buffer.

    Args:
        gps_data (pd.DataFrame): Cleaned gps data filtered out from the server for the required time window.
        bus_stops (pd.DataFrame) : Bus stops data for the trip route
        stop_buffer (int):  Radius of the buffer area to represent bus stops
        extra_buffer (int):  Extended radius of the buffer area to represent bus stops.
    
    Returns:
        bus_stops_buffer1 (GeoDataFrame) : Buffer created for filtered  Kandy-Digana direction.
        bus_stops_buffer2 (GeoDataFrame) : Buffer created for filtered  Digana-Kandy direction
        gps_data (GeoDataFrame) :  GPS data as GeoDataFrame with projected corrdinates.
        bus_stops_buffer1_add (GeoDataFrame) : Additional buffer created for filtered  Kandy-Digana direction.
        bus_stops_buffer2_add (GeoDataFrame) : Additional buffer created for filtered  Digana-Kandy direction.
  """

  #Create Geodataframe of GPS data and bus stops data
  gps_data = gpd.GeoDataFrame(gps_data, geometry=gpd.points_from_xy(gps_data.longitude,gps_data.latitude),crs='EPSG:4326')
  bus_stops = gpd.GeoDataFrame(bus_stops, geometry=gpd.points_from_xy(bus_stops.longitude,bus_stops.latitude),crs='EPSG:4326')

  #project the corrdinates in Local coordinate system
  bus_stops = bus_stops.to_crs('EPSG:5234')
  gps_data = gps_data.to_crs('EPSG:5234')

  #split bus stops dataframe into two based on route direction 
  bus_stops_direction1 = bus_stops[bus_stops['direction']=='Kandy-Digana']
  bus_stops_direction2 = bus_stops[bus_stops['direction']=='Digana-Kandy']

  bus_stops_direction2.reset_index(drop = True, inplace = True)

  #proximity analysis
  #creating a buffer
  bus_stops_buffer1 = gpd.GeoDataFrame(bus_stops_direction1, geometry = bus_stops_direction1.geometry.buffer(stop_buffer))
  bus_stops_buffer2 = gpd.GeoDataFrame(bus_stops_direction2, geometry = bus_stops_direction2.geometry.buffer(stop_buffer))

  #creating additional extra buffer to accomodate points if they were missed in standard stop buffer
  bus_stops_buffer1_add = gpd.GeoDataFrame(bus_stops_direction1, geometry = bus_stops_direction1.geometry.buffer(extra_buffer))
  bus_stops_buffer2_add = gpd.GeoDataFrame(bus_stops_direction2, geometry = bus_stops_direction2.geometry.buffer(extra_buffer))

  return bus_stops_buffer1, bus_stops_buffer2,gps_data,bus_stops_buffer1_add,bus_stops_buffer2_add

stop_buffer = 50
extra_buffer = 100
bus_stops_buffer1, bus_stops_buffer2,gps_data,bus_stops_buffer1_add,bus_stops_buffer2_add = bus_stop_buffer_create(gps_data,bus_stops,stop_buffer,extra_buffer)


In [6]:
#splitting trajectories
def bus_trajectory(gps_data,trip_ends,bus_trips):

  """
    Create bus trajectory data of sequence of bus stops with direction of trip.
    
    Args:
        gps_data (GeoDataFrame): Bus trips GPS data
        trip_ends (pd.DataFrame) : Splitted trip data from bus_trip_extraction.py
        bus_trips (pd.DataFrame) : Bus trips data
    
    Returns:
        bus_trajectory (pd.DataFrame): Sequence of bus trip trajectory data
  """
  
  #gps records that are matched with end terminals, are merged with whole GPS records
  trip_ends = trip_ends[['id','bus_stop','trip_id']]
  bus_trajectory = pd.merge(left = gps_data, right  = trip_ends,how = 'outer',left_on ='id', right_on= 'id')

  #gps records that are not associated with the terminals are asssigned as trip id = 0
  bus_trajectory["trip_id"].fillna(0, inplace = True)

  #run a loop to assign trip_id to records that are in between the terminals
  bus_trajectory.reset_index(drop = True, inplace = True)

  trip =1
  for i in range(len(bus_trajectory)-1):
    if (bus_trajectory.at[i,'trip_id']==trip) & (bus_trajectory.at[i+1, 'trip_id'] == 0):
      bus_trajectory.at[i+1,'trip_id'] = trip
    elif (bus_trajectory.at[i,'trip_id']==trip) & (bus_trajectory.at[i+1, 'trip_id'] == trip):
      trip = trip + 1
  
  bus_trajectory.drop(bus_trajectory[bus_trajectory['trip_id']==0].index, inplace = True ) #drop records that are not identified as a bus trip

  #Identify the directions of each bus trajectories using bus trips extracted data
  directions= bus_trips.set_index('trip_id').to_dict()['direction']
  bus_trajectory['direction'] = list(map(lambda x: directions[x]   ,bus_trajectory['trip_id']))

  return bus_trajectory

bus_trajectory = bus_trajectory(gps_data,trip_ends,bus_trips)

In [7]:
bus_trajectory

,id,deviceid,devicetime,latitude,longitude,speed,date,time,geometry,bus_stop,trip_id,direction
424,574670748,116,2021-10-16 07:08:31,7.299052,80.734410,7.01944,2021-10-16,07:08:31,POINT (195659.523 232979.733),BT02,1.0,2
425,574670749,116,2021-10-16 07:08:46,7.298598,80.733327,19.43850,2021-10-16,07:08:46,POINT (195539.897 232929.616),NaN,1.0,2
426,574670750,116,2021-10-16 07:09:01,7.297437,80.732405,26.45790,2021-10-16,07:09:01,POINT (195438.129 232801.161),NaN,1.0,2
427,574670751,116,2021-10-16 07:09:07,7.297405,80.731912,5.93953,2021-10-16,07:09:07,POINT (195383.650 232797.672),NaN,1.0,2
428,574670752,116,2021-10-16 07:09:22,7.297420,80.731760,11.33910,2021-10-16,07:09:22,POINT (195366.912 232799.332),NaN,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
510329,585350956,1377,2021-10-27 19:02:53,7.290715,80.638227,5.39957,2021-10-27,19:02:53,POINT (185039.303 232059.862),NaN,1365.0,2
510330,585350957,1377,2021-10-27 19:03:08,7.291095,80.637743,8.09935,2021-10-27,19:03:08,POINT (184985.951 232101.899),NaN,1365.0,2
510331,585351802,1377,2021-10-27 19:03:23,7.291273,80.637273,5.39957,2021-10-27,19:03:23,POINT (184934.062 232121.631),NaN,1365.0,2
510332,585351803,1377,2021-10-27 19:03:38,7.291202,80.636218,18.89850,2021-10-27,19:03:38,POINT (184817.571 232113.737),NaN,1365.0,2


In [8]:
bus_stops_buffer2

,stop_id,route_id,direction,address,latitude,longitude,geometry
0,BT02,654,Digana-Kandy,Digana,7.298960,80.734720,"POLYGON ((195743.751 232969.601, 195743.510 23..."
1,201,654,Digana-Kandy,Kengalla,7.291726,80.721430,"POLYGON ((194276.294 232169.792, 194276.054 23..."
2,202,654,Digana-Kandy,BOI,7.284110,80.722100,"POLYGON ((194350.152 231327.589, 194349.912 23..."
3,203,654,Digana-Kandy,Balagolla,7.287349,80.714010,"POLYGON ((193456.924 231685.871, 193456.683 23..."
4,204,654,Digana-Kandy,Pachchakaatuwa,7.284730,80.705390,"POLYGON ((192505.093 231396.387, 192504.852 23..."
5,205,654,Digana-Kandy,Pallekele (Clock Tower),7.280360,80.695670,"POLYGON ((191431.758 230913.311, 191431.518 23..."
6,206,654,Digana-Kandy,Warapitiya,7.281480,80.686720,"POLYGON ((190443.540 231037.340, 190443.299 23..."
7,207,654,Digana-Kandy,Kundasale Police College,7.280390,80.683560,"POLYGON ((190094.596 230916.872, 190094.355 23..."
8,208,654,Digana-Kandy,Nattarampotha,7.279800,80.676594,"POLYGON ((189325.389 230851.765, 189325.148 23..."
9,209,654,Digana-Kandy,Tennekumbura,7.281930,80.665940,"POLYGON ((188149.069 231087.586, 188148.828 23..."


In [9]:
bus_stops_buffer1

,stop_id,route_id,direction,address,latitude,longitude,geometry
0,BT01,654,Kandy-Digana,Kandy,7.292462,80.634978,"POLYGON ((184730.643 232253.182, 184730.402 23..."
1,101,654,Kandy-Digana,Wales Park,7.291186,80.637662,"POLYGON ((185026.961 232111.967, 185026.720 23..."
2,102,654,Kandy-Digana,Mahamaya,7.287840,80.645840,"POLYGON ((185929.853 231741.696, 185929.612 23..."
3,103,654,Kandy-Digana,Lewella junction,7.294430,80.650030,"POLYGON ((186392.694 232470.307, 186392.453 23..."
4,104,654,Kandy-Digana,Talwatta,7.286701,80.660336,"POLYGON ((187530.385 231615.322, 187530.144 23..."
5,105,654,Kandy-Digana,Tennekumbura Bridge,7.281866,80.666030,"POLYGON ((188159.005 231080.506, 188158.764 23..."
6,106,654,Kandy-Digana,Kalapura Junction Busstop,7.279830,80.676210,"POLYGON ((189283.010 230855.110, 189282.769 23..."
7,107,654,Kandy-Digana,Nattarampotha Junction Bus Stop,7.279117,80.679450,"POLYGON ((189640.749 230776.191, 189640.508 23..."
8,108,654,Kandy-Digana,Kundasale New town,7.280900,80.684160,"POLYGON ((190160.858 230973.256, 190160.617 23..."
9,109,654,Kandy-Digana,Warapitiya,7.281490,80.686350,"POLYGON ((190402.686 231038.453, 190402.445 23..."


In [10]:
def stop_buffer_filter(bus_trajectory,bus_stops_buffer1,bus_stops_buffer2,bus_stops_buffer1_add,bus_stops_buffer2_add):
    

  """

    Filter bus trip data of two buffer ranges with all the bus points, only bus stops points.
    
    Args:
        bus_trajectory (pd.DataFrame): Sequence of bus trip trajectory data
        bus_stops_buffer1 (GeoDataFrame) : Buffer created for filtered  Kandy-Digana direction.
        bus_stops_buffer2 (GeoDataFrame) : Buffer created for filtered  Digana-Kandy direction
        bus_stops_buffer1_add (GeoDataFrame) : Additional buffer created for filtered  Kandy-Digana direction.
        bus_stops_buffer2_add (GeoDataFrame) : Additional buffer created for filtered  Digana-Kandy direction.
    
    Returns:
        bus_trip_all_points (pd.DataFrame): Bus trip data with all points including null for bus_stop
        bus_stop_all_points (pd.DataFrame): Bus trip data with only bus_stops points

  """

  #project to local coordinate system before buffer filtering
  bus_trajectory = bus_trajectory.to_crs('EPSG:5234')

  #split trajectories by direction 
  trajectory_dir_1 = bus_trajectory[bus_trajectory['direction'] == 1]
  trajectory_dir_2 = bus_trajectory[bus_trajectory['direction'] == 2]

  #reset index before for loop
  trajectory_dir_1.reset_index(drop = True, inplace = True)
  trajectory_dir_2.reset_index(drop = True, inplace = True)

  #filter records within bus stops buffer of both directions
  for i in range(len(trajectory_dir_1)):
    for stop in range(len(bus_stops_buffer1)):
      if bus_stops_buffer1.iloc[stop].geometry.contains(trajectory_dir_1.iloc[i].geometry):
        trajectory_dir_1.at[i,'bus_stop'] = bus_stops_buffer1.at[stop,'stop_id']
      else:       
        if bus_stops_buffer1_add.iloc[stop].geometry.contains(trajectory_dir_1.iloc[i].geometry):
          trajectory_dir_1.at[i,'bus_stop'] = bus_stops_buffer1_add.at[stop,'stop_id']

  #filter records within bus stops buffer of both directions
  for i in range(len(trajectory_dir_2)):
    for stop in range(len(bus_stops_buffer2)):
      if bus_stops_buffer2.iloc[stop].geometry.contains(trajectory_dir_2.iloc[i].geometry):
        trajectory_dir_2.at[i,'bus_stop'] = bus_stops_buffer2.at[stop,'stop_id']
      else:       
        if bus_stops_buffer2_add.iloc[stop].geometry.contains(trajectory_dir_2.iloc[i].geometry):
          trajectory_dir_2.at[i,'bus_stop'] = bus_stops_buffer2_add.at[stop,'stop_id']    

  #concatenate dataframes of both directions and keep only records filtered within bus stops
  bus_trip_all_points = pd.concat([trajectory_dir_1,trajectory_dir_2])
  bus_stop_all_points = bus_trip_all_points.dropna()

  return bus_trip_all_points,bus_stop_all_points

bus_trip_all_points,bus_stop_all_points = stop_buffer_filter(bus_trajectory,bus_stops_buffer1,bus_stops_buffer2,bus_stops_buffer1_add,bus_stops_buffer2_add)

In [11]:
# save bus_trip_all_points,bus_stop_all_points as csv files
bus_trip_all_points.to_csv('../DataOut/bus_trip_all_points.csv',index=False)
bus_stop_all_points.to_csv('../DataOut/bus_stop_all_points.csv',index=False)

In [12]:
bus_stop_all_points

,id,deviceid,devicetime,latitude,longitude,speed,date,time,geometry,bus_stop,trip_id,direction
0,574733098,116,2021-10-16 08:03:04,7.293092,80.635573,9.17927,2021-10-16,08:03:04,POINT (184746.416 232322.760),BT01,2.0,1
5,574736662,116,2021-10-16 08:04:19,7.291640,80.637455,9.17927,2021-10-16,08:04:19,POINT (184954.136 232162.165),101,2.0,1
6,574736663,116,2021-10-16 08:04:34,7.291210,80.637713,8.63931,2021-10-16,08:04:34,POINT (184982.643 232114.617),101,2.0,1
7,574736664,116,2021-10-16 08:04:49,7.290943,80.638160,0.00000,2021-10-16,08:04:49,POINT (185031.957 232085.110),101,2.0,1
8,574736665,116,2021-10-16 08:05:04,7.290943,80.638157,0.00000,2021-10-16,08:05:04,POINT (185031.581 232085.110),101,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
153409,585349306,1377,2021-10-27 18:58:40,7.286877,80.647457,10.79910,2021-10-27,18:58:40,POINT (186058.323 231635.112),212,1365.0,2
153423,585350953,1377,2021-10-27 19:02:08,7.290202,80.639833,11.33910,2021-10-27,19:02:08,POINT (185216.691 232003.037),213,1365.0,2
153424,585350954,1377,2021-10-27 19:02:23,7.290275,80.639518,0.00000,2021-10-27,19:02:23,POINT (185181.912 232011.164),213,1365.0,2
153425,585350955,1377,2021-10-27 19:02:38,7.290398,80.638960,11.87910,2021-10-27,19:02:38,POINT (185120.271 232024.817),213,1365.0,2


In [13]:
# print raws where bus stop column is not null in bus_trip_all_points df
bus_trip_all_points[bus_trip_all_points['bus_stop'].notnull()]


,id,deviceid,devicetime,latitude,longitude,speed,date,time,geometry,bus_stop,trip_id,direction
0,574733098,116,2021-10-16 08:03:04,7.293092,80.635573,9.17927,2021-10-16,08:03:04,POINT (184746.416 232322.760),BT01,2.0,1
5,574736662,116,2021-10-16 08:04:19,7.291640,80.637455,9.17927,2021-10-16,08:04:19,POINT (184954.136 232162.165),101,2.0,1
6,574736663,116,2021-10-16 08:04:34,7.291210,80.637713,8.63931,2021-10-16,08:04:34,POINT (184982.643 232114.617),101,2.0,1
7,574736664,116,2021-10-16 08:04:49,7.290943,80.638160,0.00000,2021-10-16,08:04:49,POINT (185031.957 232085.110),101,2.0,1
8,574736665,116,2021-10-16 08:05:04,7.290943,80.638157,0.00000,2021-10-16,08:05:04,POINT (185031.581 232085.110),101,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
153409,585349306,1377,2021-10-27 18:58:40,7.286877,80.647457,10.79910,2021-10-27,18:58:40,POINT (186058.323 231635.112),212,1365.0,2
153423,585350953,1377,2021-10-27 19:02:08,7.290202,80.639833,11.33910,2021-10-27,19:02:08,POINT (185216.691 232003.037),213,1365.0,2
153424,585350954,1377,2021-10-27 19:02:23,7.290275,80.639518,0.00000,2021-10-27,19:02:23,POINT (185181.912 232011.164),213,1365.0,2
153425,585350955,1377,2021-10-27 19:02:38,7.290398,80.638960,11.87910,2021-10-27,19:02:38,POINT (185120.271 232024.817),213,1365.0,2


In [14]:
def dwell_time_estimation(bus_stop_all_points):
       
  """
    Drop terminal points from all points data. 
    Calculate arrival_time, departure_time according to check whether grouped record has 0 speed values or not.
    Dwell time derived in seconds.
    
    Args:
        bus_stop_all_points (pd.DataFrame): Bus trip data with only bus_stops points
    
    Returns:
        bus_stop_times (pd.DataFrame): Bus stops data with arrival_time, departure_time, dwell_time for each stops excluding terminals.

   """
    
  #Drop records with End Bus terminals 
  bus_stop_all_points.drop(bus_stop_all_points[bus_stop_all_points['bus_stop'] == 'BT01'].index, inplace = True)
  bus_stop_all_points.drop(bus_stop_all_points[bus_stop_all_points['bus_stop'] == 'BT02'].index, inplace = True)

  #grouping all records filtered for every bus stop
  bus_stop_all_points['grouped_ends'] = ((bus_stop_all_points['bus_stop'].shift() != bus_stop_all_points['bus_stop'])).cumsum()

  #creating a new dataframe for bus stop times
  columns = ['trip_id','deviceid','date','direction','bus_stop', 'arrival_time','departure_time','dwell_time']
  bus_stop_times = pd.DataFrame(columns=columns)

  #Loop over every grouped filtered records and choose the two records that indicate bus arrival and departure to the stop 
  for name, group in bus_stop_all_points.groupby('grouped_ends'):
    if 0 in group['speed'].values:               #if the grouped filter record has '0" speed values, then bus has stopped more than 15 seconds there and first '0'speed record as the arrival
      values = []
      trip_id = np.unique(group['trip_id'].values)[0]
      direction = np.unique(group['direction'].values)[0]
      deviceid = np.unique(group['deviceid'].values)[0]
      date = np.unique(group['date'].values)[0]
      bus_stop = np.unique(group['bus_stop'].values)[0]

      arrival_time = group[group['speed']==0]['time'].min()
      
      buffer_leaving_time = group['time'].max()
      rough_departure_time = group[group['speed']==0]['time'].max() 

      if (datetime.combine(date.min,buffer_leaving_time) - datetime.combine(date.min,rough_departure_time)).total_seconds() > 15:
        departure_time = (datetime.combine(date.min,rough_departure_time) + timedelta(seconds =15)).time()
      else:
        departure_time = buffer_leaving_time

      values.extend([trip_id,deviceid,date,direction,bus_stop,arrival_time,departure_time])
      new_row = pd.DataFrame([dict(zip(columns, values))])
      bus_stop_times = pd.concat([bus_stop_times, new_row], ignore_index=True)


    else:
      values = []
      trip_id = np.unique(group[['trip_id']].values)[0]
      direction = np.unique(group['direction'].values)[0]
      deviceid = np.unique(group[['deviceid']].values)[0]
      date = np.unique(group['date'].values)[0]
      bus_stop = np.unique(group['bus_stop'].values)[0]  

      arrival_time = group['time'].min()
      departure_time = arrival_time

      values.extend([trip_id,deviceid,date,direction,bus_stop,arrival_time,departure_time])
      new_row = pd.DataFrame([dict(zip(columns, values))])
      bus_stop_times = pd.concat([bus_stop_times, new_row], ignore_index=True)

  for i in range(len(bus_stop_times)):
    bus_stop_times.at[i,'dwell_time'] = datetime.combine(date.min,bus_stop_times.at[i,'departure_time']) - datetime.combine(date.min,bus_stop_times.at[i,'arrival_time'])

  bus_stop_times['dwell_time_in_seconds'] =  bus_stop_times['dwell_time']/np.timedelta64(1,'s')

  return bus_stop_times

bus_stop_times = dwell_time_estimation(bus_stop_all_points)

C:\Users\gw\AppData\Local\Temp\ipykernel_29272\2161691185.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_stop_all_points.drop(bus_stop_all_points[bus_stop_all_points['bus_stop'] == 'BT01'].index, inplace = True)
C:\Users\gw\AppData\Local\Temp\ipykernel_29272\2161691185.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_stop_all_points.drop(bus_stop_all_points[bus_stop_all_points['bus_stop'] == 'BT02'].index, inplace = True)
c:\Python311\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [15]:
def dwell_time_feature_addition(bus_stop_times):
    
  """
    To created aditonal derieved features for bus stops data.
    
    Args:
        bus_stop_times (pd.DataFrame): Bus stops data with arrival_time, departure_time, dwell_time for each stops excluding terminals.
    
    Returns:
        bus_stop_times (pd.DataFrame): Bus stops data with created features. 

  """
    
  #bus_stop_times = bus_stop_times.drop(bus_stop_times[bus_stop_times['dwell_time_in_seconds']>threshold].index )

  bus_stop_times['day_of_week'] = pd.to_datetime(bus_stop_times['date']).dt.weekday
  bus_stop_times['hour_of_day'] = list(map(lambda x: x.hour, (bus_stop_times['arrival_time'])))
  bus_stop_times['weekday/end'] = list(map(lambda x: 1 if x < 5 else 0 , (bus_stop_times['day_of_week'])))

  return bus_stop_times

threshold = 480
bus_stop_times = dwell_time_feature_addition(bus_stop_times)
# save as a csv file
bus_stop_times.to_csv('../DataOut/bus_stop_times.csv', index=False)

In [16]:
bus_stop_times

,trip_id,deviceid,date,direction,bus_stop,arrival_time,departure_time,dwell_time,dwell_time_in_seconds,day_of_week,hour_of_day,weekday/end
0,2.0,116,2021-10-16,1,101,08:04:49,08:05:34,0:00:45,45.0,5,8,0
1,2.0,116,2021-10-16,1,102,08:09:04,08:09:04,0:00:00,0.0,5,8,0
2,2.0,116,2021-10-16,1,103,08:16:15,08:16:15,0:00:00,0.0,5,8,0
3,2.0,116,2021-10-16,1,104,08:20:36,08:20:36,0:00:00,0.0,5,8,0
4,2.0,116,2021-10-16,1,105,08:22:21,08:22:21,0:00:00,0.0,5,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...
18040,1365.0,1377,2021-10-27,2,209,18:47:02,18:47:17,0:00:15,15.0,2,18,1
18041,1365.0,1377,2021-10-27,2,210,18:53:32,18:53:32,0:00:00,0.0,2,18,1
18042,1365.0,1377,2021-10-27,2,211,18:55:02,18:55:02,0:00:00,0.0,2,18,1
18043,1365.0,1377,2021-10-27,2,212,18:58:25,18:58:25,0:00:00,0.0,2,18,1


In [17]:
bus_stops

,stop_id,route_id,direction,address,latitude,longitude
0,BT01,654,Kandy-Digana,Kandy,7.292462,80.634978
1,101,654,Kandy-Digana,Wales Park,7.291186,80.637662
2,102,654,Kandy-Digana,Mahamaya,7.287840,80.645840
3,103,654,Kandy-Digana,Lewella junction,7.294430,80.650030
4,104,654,Kandy-Digana,Talwatta,7.286701,80.660336
5,105,654,Kandy-Digana,Tennekumbura Bridge,7.281866,80.666030
6,106,654,Kandy-Digana,Kalapura Junction Busstop,7.279830,80.676210
7,107,654,Kandy-Digana,Nattarampotha Junction Bus Stop,7.279117,80.679450
8,108,654,Kandy-Digana,Kundasale New town,7.280900,80.684160
9,109,654,Kandy-Digana,Warapitiya,7.281490,80.686350


In [18]:
bus_stops_dir_1= bus_stops[bus_stops['direction']=='Kandy-Digana']

In [19]:
bus_stops_dir_1

,stop_id,route_id,direction,address,latitude,longitude
0,BT01,654,Kandy-Digana,Kandy,7.292462,80.634978
1,101,654,Kandy-Digana,Wales Park,7.291186,80.637662
2,102,654,Kandy-Digana,Mahamaya,7.287840,80.645840
3,103,654,Kandy-Digana,Lewella junction,7.294430,80.650030
4,104,654,Kandy-Digana,Talwatta,7.286701,80.660336
5,105,654,Kandy-Digana,Tennekumbura Bridge,7.281866,80.666030
6,106,654,Kandy-Digana,Kalapura Junction Busstop,7.279830,80.676210
7,107,654,Kandy-Digana,Nattarampotha Junction Bus Stop,7.279117,80.679450
8,108,654,Kandy-Digana,Kundasale New town,7.280900,80.684160
9,109,654,Kandy-Digana,Warapitiya,7.281490,80.686350


In [20]:
# from shapely.geometry import Point

# def trip_visualization(trip_id, city_location,bus_stops_dir_1):
#     """
#     Using a  GPS data visualization package of Folium, project the coordinates on 
#     Open Street Map (OSM) to explore how the records are spread and to gain some insights and overview.
    
#     Args:
#         trip_id (int):  A random trip id to visualize its coordinates' distribution
#         city_location (arr): Longtitude and latitude of city.
#         bus_stop_dir_1 (GeoDataFrame): 
    
#     Returns:
#         map (MapObject):  A visualizable Map Object for selected trip id .
#         bus_stops_map () : 

#     """
#     geometry = [Point(xy) for xy in zip(bus_stops_dir_1['longitude'], bus_stops_dir_1['latitude'])]
#     bus_stops_dir_1 = gpd.GeoDataFrame(bus_stops_dir_1, geometry=geometry)
#     trip = bus_trajectory[bus_trajectory['trip_id']==trip_id]
#     trip = trip.to_crs('EPSG:4326')
#     bus_stops_dir_1 = bus_stops_dir_1.to_crs('EPSG:4326')

#     map = folium.Map(location=city_location, tiles='openstreetmap', zoom_start=14)
#     for idx, row in trip.iterrows():
#         Marker([row['latitude'], row['longitude']]).add_to(map)

#     bus_stops_map = folium.Map(location=city_location, tiles='openstreetmap', zoom_start=14)
#     for idx, row in bus_stop_dir_1.iterrows():
#         Marker([row['latitude'], row['longitude']]).add_to(bus_stops_map)
    
#     return map, bus_stops_map

# trip_id = 1560  #Choose a random trip id to visualize its coordinates' distribution
# city_location = [7.2906,80.6337] #kandy city location

# map, bus_stops_map = trip_visualization(trip_id,city_location,bus_stops_dir_1)
